In [17]:
import os
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
MODEL_DIR = '../models'
LOG_DIR = '../logs'
SIZE = (224, 224)
caminho_bases = os.path.join('..', 'bases', 'vazios')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

# Callbacks

In [18]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                              verbose=1, min_delta=1e-2, mode='min')


# Model

In [19]:
from  tensorflow.keras.models import Model
from  tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from  tensorflow.keras import backend as K

# create the base pre-trained model
base_model = DenseNet121(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=output)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional DenseNet121 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

# Training and validation

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
)
train_generator = train_datagen.flow_from_directory(
    caminho_train,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
)

validation_generator = validation_datagen.flow_from_directory(
    caminho_test,
    target_size=SIZE,
    batch_size=32,
    class_mode='binary'
)

Found 20845 images belonging to 2 classes.
Found 2317 images belonging to 2 classes.


In [14]:
epoch=6
val_loss=0.63
model.load_weights(
    os.path.join(MODEL_DIR,
                 'Transfermodelweights.{:02d}-{:.2f}.hdf5'.format(epoch, val_loss)
                ))


In [21]:
history = model.fit_generator(train_generator,
                              initial_epoch=epoch,
                              epochs=20,
                              verbose=1,
                              callbacks=[early_stop, mcp_save, reduce_lr, tensorboard_logs],
                              validation_data=validation_generator)

Epoch 11/20
326/326 [==============================] - 307s 942ms/step - loss: 0.2482 - acc: 0.9182 - val_loss: 0.8626 - val_acc: 0.5870
Epoch 12/20
326/326 [==============================] - 291s 894ms/step - loss: 0.1729 - acc: 0.9442 - val_loss: 0.4687 - val_acc: 0.7799
Epoch 13/20
326/326 [==============================] - 292s 895ms/step - loss: 0.1600 - acc: 0.9487 - val_loss: 0.3712 - val_acc: 0.8334
Epoch 14/20
326/326 [==============================] - 292s 897ms/step - loss: 0.1499 - acc: 0.9515 - val_loss: 0.4375 - val_acc: 0.7915
Epoch 15/20
325/326 [============================>.] - ETA: 0s - loss: 0.1482 - acc: 0.9524
Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
326/326 [==============================] - 296s 909ms/step - loss: 0.1488 - acc: 0.9523 - val_loss: 0.6346 - val_acc: 0.6452
Epoch 16/20
326/326 [==============================] - 292s 895ms/step - loss: 0.1290 - acc: 0.9570 - val_loss: 0.4643 - val_acc: 0.7924
Epoch 17/20
325/3

# Fine tunning

In [23]:
print(len(model.layers))
for i, layer in enumerate(model.layers):
    print(i, layer.name)

430
0 input_3
1 zero_padding2d_4
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_5
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_b

In [24]:
frozen_layers = 312

In [29]:
for layer in model.layers[:frozen_layers]:
    layer.trainable = False
for layer in model.layers[frozen_layers:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

In [30]:
epoch_tunning = 0

In [ ]:
mcp_save2 = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'TransfermodelTunningweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
history = model.fit_generator(train_generator,
                              initial_epoch=epoch_tunning,
                              epochs=20,
                              verbose=1,
                              callbacks=[early_stop, mcp_save2, reduce_lr, tensorboard_logs],
                              validation_data=validation_generator)

Epoch 1/20
 67/326 [=====>........................] - ETA: 22:49 - loss: 0.1326